# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,32.18,85,88,10.16,GS,1716056601
1,1,clovis,36.8252,-119.7029,82.62,51,40,4.61,US,1716056896
2,2,ust-nera,64.5667,143.2000,39.54,99,100,3.65,RU,1716056644
3,3,hami,42.8000,93.4500,80.26,17,100,3.47,CN,1716057134
4,4,iskateley,67.6803,53.1512,35.02,92,100,21.00,RU,1716057134


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [13]:
%%capture --no-display

# Configure the map plot
map = city_data_df.hvplot.points(x='Lng', y='Lat', geo=True, size='humidity', alpha=0.7, color='blue', hover_cols=['humidity'])

# Display the map

# Display the map
map

:Points   [Lng,Lat]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:
# Narrow down cities that fit criteria and drop any results with null values
cities_filtered = city_data_df.loc[(city_data_df['Max Temp'] <= 27) & (city_data_df['Max Temp'] >= 22)]
cities_filtered = cities_filtered.loc[(city_data_df['Wind Speed'] >= 4.5)]
cities_filtered = cities_filtered.loc[(city_data_df['Cloudiness'] >= 0)]


# Drop any rows with null values
cities_filtered.dropna()

# Display sample data
cities_filtered

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
25,25,longyearbyen,78.2186,15.6401,24.64,86,100,12.66,SJ,1716056643
302,302,nadym,65.5333,72.5167,25.97,86,20,12.71,RU,1716057290
515,515,tarko-sale,64.9225,77.7850,24.10,89,56,11.81,RU,1716057137


### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame(city_data_df, columns=["City", "Country", "Lat","Lng", "Humidity"])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,grytviken,GS,-54.2811,-36.5092,85,
1,clovis,US,36.8252,-119.7029,51,
2,ust-nera,RU,64.5667,143.2000,99,
3,hami,CN,42.8000,93.4500,17,
4,iskateley,RU,67.6803,53.1512,92,
...,...,...,...,...,...,...
567,pasighat,IN,28.0667,95.3333,73,
568,polyarnyy,RU,69.1989,33.4478,99,
569,lichuan,CN,30.3000,108.8500,97,
570,kemin,KG,42.7846,75.6892,93,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [27]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
#conditions = "vegetarian"

# Set the parameters for the type of search
filters = f"circle:{lng},{lat},{radius}"
bias = f"proximity:{lng},{lat}"
limit = 1

params = {
    "categories":categories,
    #"conditions":conditions,
    "limit":limit,
    "filter":filters,
    "bias":bias,
    "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = ["Lat"]
    lng = ["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'latitude={lat},longitude={lng}'
    params["bias"] = f'latitude={lat},longitude={lng}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
grytviken - nearest hotel: No hotel found
clovis - nearest hotel: No hotel found
ust-nera - nearest hotel: No hotel found
hami - nearest hotel: No hotel found
iskateley - nearest hotel: No hotel found
sao jose da coroa grande - nearest hotel: No hotel found
kudahuvadhoo - nearest hotel: No hotel found
jamestown - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
mount pearl - nearest hotel: No hotel found
bonthe - nearest hotel: No hotel found
port-aux-francais - nearest hotel: No hotel found
kerikeri - nearest hotel: No hotel found
tazacorte - nearest hotel: No hotel found
isafjordur - nearest hotel: No hotel found
novokayakent - nearest hotel: No hotel found
illela - nearest hotel: No hotel found
bilibino - nearest hotel: No hotel found
cabo san lucas - nearest hotel: No hotel found
arraial do cabo - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
novo aripuana - nearest hotel: No hotel found
avarua

,City,Country,Lat,Lng,Humidity,Hotel Name
0,grytviken,GS,-54.2811,-36.5092,85,No hotel found
1,clovis,US,36.8252,-119.7029,51,No hotel found
2,ust-nera,RU,64.5667,143.2000,99,No hotel found
3,hami,CN,42.8000,93.4500,17,No hotel found
4,iskateley,RU,67.6803,53.1512,92,No hotel found
...,...,...,...,...,...,...
567,pasighat,IN,28.0667,95.3333,73,No hotel found
568,polyarnyy,RU,69.1989,33.4478,99,No hotel found
569,lichuan,CN,30.3000,108.8500,97,No hotel found
570,kemin,KG,42.7846,75.6892,93,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [29]:
%%capture --no-display

# Configure the map plot
map2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    color = "City",
    size = "Humidity",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ['Country','Hotel Name'],
    tiles = "OSM"
)

# Display the map
map2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)